$\textbf{INTRO}$

Nello scaling dei sistemi di QC c'é concentrazione exp. della Loss Function in funzione dei parametri(Barren Plateu). Ciò significa che nello spazio dei parametri, la funzione di loss ha un gradiente che va come un esponenziale. Dunque tende a essere molto piatto in tutto lo spazio dei parametri e a convergere iper-velocemente in un unico punto. Quando è presente rumore, la LF non può essere descritta in PQC, con la notazione standard di Lie.

VQC $\rightarrow$ iterativamente ottimizzazione di PQC $\rightarrow$ Minimizzare $\text{Loss}$ dato $\hat{\rho}$.

BP: appiattimento exp. della LF $\rightarrow$ ottimizzazione impossibile data una loss function classica. 

In ultima analisi, il problema sorge dalla dimensionalità (la small angle initialization è una soluzione comune). I BP possono essere indotti anche da noise ambientale e decoerenza (NIBP).


$\textbf{NOTAZIONE}$


$ \textit{H} = \otimes_{m} H_m$ , $dim(\mathcal{H}) = \Pi_m dim(\mathcal{H}_m)$, con $\mathcal{H}_m$ indichiamo un Q-Bit o un gruppo di qbit.

Indichiamo il canale quantistico che agisce sui Q-bit $\Phi_{\vec{\theta}}$. La funzione di costo che viene minimizzata è $\mathcal{L}_{\rho, k}=Tr[\Phi(\rho)H]$, con H osservabile. Il BP può essere visto dalla varianza di $\mathcal{L}: \textbf{V}_{\rho,H}(\theta)$. Nello specifico, c'è un BP se $\textbf{V} \propto O(e^{-\beta n})$ con n numero di QBIT.

Per un sistema layered: $\Phi_{\theta}= \mathcal{U_{L+1} \circ E_L \circ U_{L-1} \circ ... \circ E_1}$, dove $\mathcal{U_i}$ sono operatori unitari e $\mathcal{E_i}$ operatori qualsiasi, chiamati "strettamente contrattivi" se($\|E_i\| < 1$).

Se $\mathcal{B}$ è lo spazio degli operatori limitati su $\mathcal{H}$, tutte le componenti dei sistemi a strati mappano \mathcal{B} in se stesso. Inoltre \mathcal{B} preserva la suddivisione in sottosistemi di \mathcal{H}.

??? 

Diremo che un sottospazio A $\in \mathcal{B_k}$ è k-locale se il vettore di località $l_A \in {R^{2^m}}$ : $(l_A)_k = \sum_{j=i}^{k} Tr[B_j A]^2$. $d_k=dim(\mathcal{B}_k)= \Pi_m (d^2_m-1)$, i $\mathcal{B}_j$ sono una base di $\mathcal{B}_k$.
La matrice di trasferimento locale per $\Lambda: \mathcal{B} \rightarrow \mathcal{B}$ è $T_{k,\lambda} = \frac{1}{d_k}\sum_{j=1}^{d_k}(l_{\Lambda (B_j)})_\lambda $, se $\Lambda$ preserva la località, $T= I$. In questo senso, gli $\mathcal{U}$ sono locali, $\mathcal{E}$ no.
Per via dell'entanglemet, che causa decorenza, un QC può essere estremamente efficiente in un sottosistema, ma inefficiente nel complessivo.
Vale $ TD=(T^{+} D)^t$, con $D_{k,l} = \delta_{k,l}d_k$. Così definiamo il prodotto scalare in $R^{2^M}$: $(a,b)=a^t D^{-1} b=\sum_{k} \frac{a_k b_k }{d_k}$

$\textbf{RISULTATI}$

L è il numero dei layers. $E_{\theta}{.}$ rappresenta il valore di aspettazione di . rispetto al parametro $\theta$.
Vale: $E_{\theta}(Tr[\mathcal{U}_{\theta}(\rho)H]^2) = (l_{\rho},l_H)$. Dunque, termini di località diversa non contribuiscono insieme alla varianza.
In generale: $E_{\theta}(Tr[\Phi_{\theta}(\rho)H]^2) = (l_{\rho}, \Pi_{l=1}^{L}T_l l_H)$, da cui: 
$\textbf{V}_{\rho,H}^L = (l_{\rho}, \Pi_{l=1}^{L}T_l l_H) -\frac{Tr[H]^2}{d^2}$ .

$\textbf{CIRCUITI "PROFONDI"}$




$\textbf{CIRCUITI "PICCOLI"}$



$\textbf{NOYSE SIMULATION}$

Per simuare CQ reali è necessario aggiungere rumore. 
Un modo di parlo è randomizzare classicamente parametri: ad esempio $O(\theta)= <0|U(\theta)^+ O U(\theta|0>)$ prendendo $\theta$ parametro casuale.
Un altro modo è usare il bitflip. $\rho \rightarrow \Phi(\rho)= \sum_i K_i \rho K_i^+$ con $\sum_i K_i K_i^+ =I$. Se utilizziamo $K_0 = \sqrt{1-p}I, K_1=\sqrt{p}X$. Con una probabilità p un beat viene mandato nel suo strato opposto.